In [119]:
import numpy as np
import cv2 as cv
import math

In [120]:
img = cv.imread('hough.jpg',0)
[rows,cols] = img.shape

# Padding

In [121]:
def padding(img):
    [rows,cols] = img.shape
    listofzeros = [0] * (cols+2)
    image_pad = list()
    image_pad.append(listofzeros)
    for i in img:
        i = list(i)
        i.insert(0,0)
        i.insert(len(i)+1,0)
        image_pad.append(i)
    image_pad.append(listofzeros)
    image_pad = np.asarray(image_pad)
    return image_pad

# Convolution

In [122]:
def convolution(image,kernel):
    [rows,cols] = image.shape
    image_pad = padding(image)
    conv_img = [[0] * (int)(cols) for _ in range((int)(rows))]
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            s=0
            for k in range(3):
                for l in range(3):
                    s+=image_pad[i-1+k,j-1+l]*kernel[k,l]
            conv_img[i-1][j-1] = s
    return np.asarray(conv_img)

# Edge detection

In [123]:
def edges(conv_img,threshold):
    for i in range(conv_img.shape[0]):
        for j in range(conv_img.shape[1]):
            if conv_img[i,j]>threshold:
                conv_img[i,j] = 255
            else:
                conv_img[i,j] = 0
    return conv_img

In [124]:
x_sob = [[1, 0, -1], 
         [2, 0, -2], 
         [1, 0, -1]]
y_sob = [[1, 2, 1], 
         [0, 0, 0], 
         [-1, -2, -1]]
x_sob = np.asarray(x_sob)
y_sob = np.asarray(y_sob)

In [125]:
conv_img_y = convolution(img,y_sob)
conv_img_y = edges(conv_img_y,150)
cv.imwrite('y_sob_bonus.png',conv_img_y)

True

In [126]:
conv_img_x = convolution(img,x_sob)
conv_img_x = edges(conv_img_x,150)
cv.imwrite('x_sob_bonus.png',conv_img_x)

True

In [127]:
conv_img = np.empty([rows,cols])
for i in range(rows):
    for j in range(cols):
        conv_img[i,j] = math.sqrt(conv_img_x[i,j]**2 + conv_img_y[i,j]**2)
cv.imwrite('bonus_edges.png',conv_img)

True

# Hough transform


In [160]:
R=22
def find_rho(a1,b1,t):
    
    a = int((b1 - R * math.cos(math.radians(t))))
    b = int((a1 - R * math.sin(math.radians(t))))
    
    return a,b

# Accumulator matrix

In [161]:
def accumulator_matrix(conv):
    accumulator = np.zeros((2*rows,2*cols))
    for i in range(conv.shape[0]):
        for j in range(conv.shape[1]):
            if conv[i,j] == 255:
                for k in range(0,360):
                    a,b = find_rho(i,j,k)
                    accumulator[a,b] = accumulator[a,b] + 1
    return accumulator

In [163]:
accumulator_circle2 = accumulator_matrix(conv_img)
cv.imwrite('accumulator_circle.png',accumulator_circle2)

True

# Finding peaks in the sinusoidal curve

In [165]:
def max_indices(arr, k):
    '''
    Returns the indices of the k first largest elements of arr
    (in descending order in values)
    '''
    assert k <= arr.size, 'k should be smaller or equal to the array size'
    arr_ = arr.astype(float)  # make a copy of arr
    max_idxs = []
    for _ in range(k):
        max_element = np.max(arr_)
        if np.isinf(max_element):
            break
        else:
            idx = np.where(arr_ == max_element)
        max_idxs.append(idx)
        arr_[idx] = -np.inf
    return np.asarray(max_idxs)

In [166]:
def points(indices):
    k = []
    for i in range(indices.shape[0]):
        n = indices[i][0].shape[0]
        if n>1:
            for j in range(n):
                k.append([indices[i][0][j], indices[i][1][j]])
        else:
            k.append([indices[i][0][0], indices[i][1][0]])
    return np.asarray(k)

In [182]:
indices_circle = max_indices(accumulator_circle2,43)
indices_circle = points(indices_circle)
indices_circle.shape

(97, 2)

In [183]:
def plotting_circles(conv,indices,r,g,b):
    img = cv.imread('hough.jpg')
    coord = []
    for k in range(indices.shape[0]):
        a,b = indices[k,0],indices[k,1]
        cv.circle(img,(a,b),R,(0,255,0))
    cv.imwrite('output_circles.png',img)

In [184]:
plotting_circles(conv_img,indices_circle,255,255,0)